In [1]:
import sys
import json 
import joblib
import gc
from tqdm import tqdm
import os
from typing import List, Dict, Tuple
import yaml

# TO CHANGE
BASEDIR = "../"
sys.path.insert(0, BASEDIR)

from src.pipelines.qa.kg_reasoning.weak_reasoner import WeakKGReasoner, WeakKGReasonerConfig

/home/dzigen/Desktop/PersonalAI/.pai_venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
from src import GraphModelConfig, EmbeddingsModelConfig, EmbedderModelConfig

from src.db_drivers import KeyValueDriverConfig, GraphDriverConfig, VectorDriverConfig
from src.db_drivers.kv_driver import DEFAULT_INMEMORYKV_CONFIG
from src.db_drivers.graph_driver import DEFAULT_INMEMORYGRAPH_CONFIG
from src.db_drivers.vector_driver import VectorDBConnectionConfig

from src.pipelines.memorize import LLMExtractorConfig, LLMUpdatorConfig, MemPipelineConfig, MemPipeline
from src.pipelines.qa.kg_reasoning.weak_reasoner.knowledge_retriever import KnowledgeRetrieverConfig, NaiveGraphSearchConfig
from src.kg_model import KnowledgeGraphModel

from src.utils import NodeType, Logger

In [3]:
# Graph model configuration
GRAPH_STORAGE_CONFIG = GraphDriverConfig(db_vendor='inmemory_graph', db_config=DEFAULT_INMEMORYGRAPH_CONFIG)
GRAPH_MODEL_CONFIG = GraphModelConfig(driver_config=GRAPH_STORAGE_CONFIG)

# Vector model configuration
NODES_DB_PATH = f'{BASEDIR}/data/graph_structures/vectorized_nodes/testing'
TRIPLETS_DB_PATH = f'{BASEDIR}/data/graph_structures/vectorized_triplets/testing'
NEED_TO_CLEAR = True

VECTOR_NODES_STORAGE_CONFIG = VectorDriverConfig(db_config=VectorDBConnectionConfig(path=NODES_DB_PATH, need_to_clear=NEED_TO_CLEAR))
VECTOR_TRIPLETS_STIRAGE_CONFIG = VectorDriverConfig(db_config=VectorDBConnectionConfig(path=TRIPLETS_DB_PATH, need_to_clear=NEED_TO_CLEAR))

DEVICE = 'cuda'
EMBEDDER_MODEL_PATH = f'{BASEDIR}/models/intfloat/multilingual-e5-small'
EMBEDDER_MODEL_CONFIG = EmbedderModelConfig(model_name_or_path=EMBEDDER_MODEL_PATH, device=DEVICE)

VECTOR_MODEL_CONFIG = EmbeddingsModelConfig(
    nodesdb_driver_config=VECTOR_NODES_STORAGE_CONFIG,
    tripletsdb_driver_config=VECTOR_TRIPLETS_STIRAGE_CONFIG,
    embedder_config=EMBEDDER_MODEL_CONFIG)

In [4]:
# Memorize-pipeline configuration
DELETE_OBSOLETE_INFO = False

MEM_PIPELINE_CONFIG = MemPipelineConfig(
    extractor_config=LLMExtractorConfig(lang='en'),
    updator_config=LLMUpdatorConfig(lang='en', delete_obsolete_info=DELETE_OBSOLETE_INFO))

In [5]:
kg_model = KnowledgeGraphModel(
    graph_config=GRAPH_MODEL_CONFIG,
    embeddings_config=VECTOR_MODEL_CONFIG)

No sentence-transformers model found with name ..//models/intfloat/multilingual-e5-small. Creating a new one with mean pooling.


In [6]:
kg_model.count_items()

{'graph_info': {'triplets': 0, 'nodes': 0},
 'embeddings_info': {'nodes': 0, 'triplets': 0}}

In [7]:
mem_pipeline = MemPipeline(kg_model=kg_model, config=MEM_PIPELINE_CONFIG)

In [8]:
messages = [
    "Mikhail Menshchikov is currently a second-year master's student at ITMO.",
    "Mikhail Menshchikov is studying in the Master's program 'Deep Learning and Generative AI'",
    "Mikhail Menshchikov completed his bachelor's degree at Petrozavodsk State University",
    "Petrozavodsk State University is where Mikhail Menshchikov received his bachelor's degree.",
    "Mikhail Menshchikov studied at Petrozavodsk State University and received a bachelor's degree."]
properties = [dict() for _ in range(len(messages))]

In [9]:
for msg, prop in zip(messages, properties):
    triplets, info = mem_pipeline.remember(msg, prop)

In [13]:
reasoner_config = WeakKGReasonerConfig(
    query_parser_config=None,
    knowledge_comparator_config=None,
    knowledge_retriever_config=KnowledgeRetrieverConfig(
        retriever_method='naive_retriever',
        retriever_config=NaiveGraphSearchConfig(max_k=2),
        filter_method=None
    )
)

In [14]:
reasoner = WeakKGReasoner(kg_model, reasoner_config)

In [16]:
reasoner.perform("Where did D.J. Korzun born?")

("Chain of thought: 1. The provided information does not specify where D.J. Korzun was born. 2. It includes two facts: he obtained his bachelor's degree at Petrozavodsk State University, and Mikhail Menshchikov also completed his bachelor's degree. 3. The location mentioned is Petrozavodsk, which is in Russia.\n\nFinal answer: Petrozavodsk, Russia",
 ReturnInfo(occurred_warning=[], status=<ReturnStatus.success: 0>, message=''))